In [ ]:
import Data.List

In [ ]:
data Op = Plus | Minus | Mul | Div | Pow
    deriving (Eq, Show)
    
data SymbolicManip a =
    Number a
  | Symbol String
  | BinaryArith Op (SymbolicManip a) (SymbolicManip a)
  | UnaryArith String (SymbolicManip a)
    deriving (Eq, Show)

In [ ]:
data Units a = Units a (SymbolicManip a)
    deriving (Eq)

In [ ]:
instance Num a => Num (SymbolicManip a) where
    a + b = BinaryArith Plus a b
    a - b = BinaryArith Minus a b
    a * b = BinaryArith Mul a b
    negate = BinaryArith Mul (Number (-1))
    abs = UnaryArith "abs"
    signum a = error "signum is unimplemented"
    fromInteger i = Number (fromInteger i)

In [ ]:
instance (Num a) => Num (Units a) where
    (Units xa ua) + (Units xb ub)
        | ua == ub 

In [ ]:
instance (Fractional a) => Fractional (SymbolicManip a) where
    a / b = BinaryArith Div a b
    recip = BinaryArith Div (Number 1)
    fromRational r = Number (fromRational r)

In [ ]:
instance (Floating a) => Floating (SymbolicManip a) where
    pi = Symbol "pi"
    exp = UnaryArith "exp"
    log = UnaryArith "log"
    sqrt =  UnaryArith "sqrt"
    a ** b = BinaryArith Pow a b
    sin = UnaryArith "sin"
    cos = UnaryArith "cos"
    tan = UnaryArith "tan"
    asin = UnaryArith "asin"
    acos = UnaryArith "acos"
    atan = UnaryArith "atan"
    sinh = UnaryArith "sinh"
    cosh = UnaryArith "cosh"
    tanh = UnaryArith "tanh"
    asinh = UnaryArith "asinh"
    acosh = UnaryArith "acosh"
    atanh = UnaryArith "atanh"

In [ ]:
instance Show Op where
    show Plus = "+"
    show Minus = "-"
    show Mul = "*"
    show Div = "/"
    show Pow = "**"

In [ ]:
instance (Show a, Num a) =>  Show (SymbolicManip a) where
    show = prettyShow

In [ ]:
prettyShow :: (Show a, Num a) => SymbolicManip a -> String

prettyShow (Number x) = show x
prettyShow (Symbol x) = x

prettyShow (BinaryArith op a b) = pa ++ show op ++ pb
    where
        pa = simpleParen a
        pb = simpleParen b
        
prettyShow (UnaryArith opstr a) =
    opstr ++ "(" ++ show a ++ ")"
    
simpleParen :: (Show a, Num a) => SymbolicManip a -> String

simpleParen (Number x) = prettyShow (Number x)
simpleParen (Symbol x) = prettyShow (Symbol x)
simpleParen x@(BinaryArith _ _ _) = "(" ++ prettyShow x ++ ")"
simpleParen x@(UnaryArith _ _) = prettyShow x

In [ ]:
rpnShow :: (Show a, Num a) => SymbolicManip a -> String

rpnShow i = join " " (toList i)
    where
        toList (Number x) = [show x]
        toList (Symbol x) = [x]
        toList (BinaryArith op a b) = toList a ++ toList b ++ [show op]
        toList (UnaryArith op a) = toList a ++ [op]
        join :: [a] -> [[a]] -> [a]
        join delim l = concat (intersperse delim l)

In [ ]:
simplify :: (Num a, Eq a) => SymbolicManip a -> SymbolicManip a

simplify (BinaryArith op ia ib) = 
    case (op, sa, sb) of
        (Mul, Number 1, b) -> b
        (Mul, a, Number 1) -> a
        (Mul, Number 0, _) -> Number 0
        (Mul, _, Number 0) -> Number 0
        (Div, a, Number 1) -> a
        (Plus, a, Number 0) -> a
        (Plus, Number 0, b) -> b
        (Minus, a, Number 0) -> a
        _ -> BinaryArith op sa sb
    where
        sa = simplify ia
        sb = simplify ib
simplify (UnaryArith op a) = UnaryArith op (simplify a)
simplify x = x

In [ ]:
zero = Number 0
one = Number 1
five = Number 5
seven = Number 7
ten = Number 10
m = (five * (ten * one) + (five * zero) + zero) / seven / one
print m
print $ simplify m